# Relax, Vibrations, and Phonons

Import useful modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#
from ase.io import read
from ase.build import molecule, bulk
from ase.visualize import view
from ase.calculators.espresso import Espresso, EspressoProfile
from ase.calculators.emt import EMT

Legacy setup of ASE environment variable for running Quantum Espresso. NOTE: the qepath needs to be changed to the installation folder of QE.

In [ ]:
#import os
# runcommand = "mpirun -np 4"
# qepath = "/Users/oliviero/PWSCF/espresso-git/bin/"
#os.environ['ASE_ESPRESSO_COMMAND'] = runcommand+qepath+"pw.x -in PREFIX.pwi > PREFIX.pwo"
#os.environ['OMP_NUM_THREADS'] = "1"

The following should be the most uptodate approach to setup the QE calculator variables.

In [ ]:
# Optionally create profile to override paths in ASE configuration:
profile = EspressoProfile(
    command='mpirun -np 4 /Users/oliviero/PWSCF/espresso-git/bin/pw.x', pseudo_dir='./pseudos/'
)

Unit conversion factors

In [ ]:
eV2Ry = 13.605662285137 # energy conversion factor
eV2kcal_mol = 23.0609 # energy conversion factor
bohr2ang = 0.5291772 # length conversion factor
ang2bohr = 1./bohr2ang

## Geometry Optimization of a Water Molecule

We can use the `molecule` function of ASE to generate a water molecule

In [ ]:
atoms = molecule('H2O')
view(atoms, viewer="x3d")

In order to simulate the molecule with QE, we need to setup the simulation cell. In this case, a cubic box of side 15 Angstrom and periodic boundary conditions. 

In [ ]:
atoms.set_cell(15. * np.identity(3))
atoms.set_pbc((True, True, True))
atoms.center()

Setup pseudopotentials information

In [ ]:
pseudopotentials = {
    "H":"H.pbe-rrkjus.UPF",
    "O":"O.pbe-rrkjus.UPF"
}

Setup of QE input parameters and calculator

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': '/Users/oliviero/PWSCF/espresso-git/pseudo',
        'calculation': 'relax',
        'prefix': 'H2O_vacuum'
    },
    'system': {
        'ecutwfc': 30,
        'ecutrho': 300
    },
    'electrons': {
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.7
    }
} 

calc = Espresso(
    profile=profile,
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

atoms.calc = calc
# atoms.calc = EMT()

The `get_potential_energy()` methods of the `Atoms` object will call the calculator and perform the DFT simulation. 

In [ ]:
energy = atoms.get_potential_energy()
print(f"Energy in vacuum = {energy:.3f} eV")

We can now use ASE to read the output file and use the relaxed structure.

In [ ]:
relaxed_atoms = read('./espresso.pwo',format='espresso-out')

In [ ]:
print(atoms.positions)
print(relaxed_atoms.positions)

# Use ASE for Geometry Optimization

the pw.x tool of QE mostly relies on BFGS for geometry relaxation. In ASE there is a separate implementation of the BFGS algorithm, as well as other optimization tools. We can thus submit QE calculations that only perform the SCF, and use ASE to control the geometry relaxation. 

In [ ]:
from ase.optimize import BFGS

Re-perform the setup of the previous system

In [ ]:
atoms = molecule('H2O')
atoms.set_cell(15. * np.identity(3))
atoms.set_pbc((True, True, True))
atoms.center()

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': './pseudos/',
        'calculation': 'scf',
        'prefix': 'H2O_vacuum'
    },
    'system': {
        'ecutwfc': 30,
        'ecutrho': 300
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.4
    }
} 

calc = Espresso(
    profile=profile,
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

atoms.calc = calc
#atoms.calc = EMT()

In [ ]:
optimization = BFGS(atoms,trajectory='H2O-opt.traj')
optimization.run(fmax=0.001)

In [ ]:
np.sqrt(np.sum((atoms.positions[0,:]-atoms.positions[1,:])**2))

In [ ]:
np.sqrt(np.sum((relaxed_atoms.positions[0,:]-relaxed_atoms.positions[1,:])**2))

## Vibrations with ASE

Once we have a relaxed structure, we can use finite differences to compute the Hessian matrix and diagonalize it to obtain the vibrational frequencies and vibrational modes of the atomistic system. In the following example, we will use the EMT calculator instead of QE

In [ ]:
from ase.vibrations import Vibrations

In [ ]:
atoms.calc = EMT()

The summary contains information on the vibrational modes and the zero point energy

In [ ]:
vib = Vibrations(atoms)
vib.run()
vib.summary(log='H2O_vib_summary.txt')

## Use ASE for Phonons Calculations

Once we have a relaxed structure, we can use finite differences to compute the Hessian matrix and diagonalize it to obtain the vibrational frequencies and vibrational modes of the atomistic system. In the following example, we will use the EMT calculator instead of QE

In [ ]:
from ase.phonons import Phonons

In [ ]:
# Setup crystal and EMT calculator
atoms = bulk('Al', 'fcc', a=4.05)

In [ ]:
# Phonon calculator
N = 7
ph = Phonons(atoms, EMT(), supercell=(N, N, N), delta=0.05)
ph.run()

In [ ]:
ph.read(acoustic=True)
ph.clean()

In [ ]:
ph.C_N

In [ ]:
path = atoms.cell.bandpath('GXULGK', npoints=100)
bs = ph.get_band_structure(path)

In [ ]:
dos = ph.get_dos(kpts=(20, 20, 20)).sample_grid(npts=100, width=1e-3)


In [ ]:
# Plot the band structure and DOS:
import matplotlib.pyplot as plt  # noqa

fig = plt.figure(1, figsize=(7, 4))
ax = fig.add_axes([.12, .07, .67, .85])

emax = 0.035
bs.plot(ax=ax, emin=0.0, emax=emax)

dosax = fig.add_axes([.8, .07, .17, .85])
dosax.fill_between(dos.get_weights(), dos.get_energies(), y2=0, color='grey',
                   edgecolor='k', lw=1)

dosax.set_ylim(0, emax)
dosax.set_yticks([])
dosax.set_xticks([])
dosax.set_xlabel("DOS", fontsize=18)


In [ ]:
dos.get_energies()

In [ ]:
vib.show_as_force(8)